## CASE

SQL's verision of IF this THEN that.

```
SELECT 
    id,
    home_goal,
    away_goal,
    CASE WHEN x = 1 THEN 'a'
         WHEN x = 2 THEN 'b'
         ELSE 'c' END AS new_column
FROM match
WHERE season = '2013/2014';
```

We can also add multiple logical conditions to the WHEN clause using AND:

```
SELECT date, hometeam_id, awayteam_id,
    CASE WHEN hometeam_id = 8455 AND home_goal > away_goal
         THEN 'Chelsea home win!'
    WHEN awayteam_id = 8455 AND home_goal < away_goal
         THEN 'Chelsea away win!'
    ELSE 'Loss or tie :(' END AS outcome
FROM match
WHERE hometeam_id = 8455 OR awayteam_id = 8455;
```

Filtering using WHERE to not display NULL results:
```
SELECT date, hometeam_id, awayteam_id,
    CASE WHEN hometeam_id = 8455 AND home_goal > away_goal
         THEN 'Chelsea home win!'
    WHEN awayteam_id = 8455 AND home_goal < away_goal
         THEN 'Chelsea away win!'
FROM match
WHERE CASE WHEN hometeam_id = 8455 AND home_goal > away_goal
           THEN 'Chelsea home win!'
      WHEN awayteam_id = 8455 AND home_goal < away_goal
           THEN 'Chelsea away win!' END IS NOT NULL;
```

CASE with Aggregations:
```
SELECT
    season,
    COUNT(CASE WHEN hometime_id=8650 AND home_goal > away_goal
          THEN 54321 END) AS home_wins
FROM match
GROUP BY season;
```

## Subqueries

A query nested inside another query.  Can be put in any part of a query.

```
SELECT col
FROM (SELECT col
      FROM table) AS subquery;
```

```
SELECT 
	-- Select the stage and average goals from s
	stage,
    ROUND(avg_goals,2) AS avg_goal,
    -- Select the overall average for 2012/2013
    (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
	-- Select the stage and average goals in 2012/2013 from match
	(SELECT
		 stage,
         AVG(home_goal + away_goal) AS avg_goals
	 FROM match
	 WHERE season = '2012/2013'
	 GROUP BY stage) AS s
WHERE 
	-- Filter the main query using the subquery
	s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                    FROM match WHERE season = '2012/2013');
```

## Comments

Use `/* TEXT HERE */` for block comments.

Use -- for in line comments.

## Correlated Subqueries
Uses values from the outer query to generate results where the subquery is re-run for every row generated.  Cannot be executed on its own since it it dependent on the main query.

```
SELECT 
	-- Select country ID, date, home, and away goals from match
	main.country_id,
    main.date,
    main.home_goal,
    main.away_goal
FROM match AS main
WHERE 
	-- Filter for matches with the highest number of goals scored
	(home_goal + away_goal) > 
        (SELECT MAX(sub.home_goal + sub.away_goal)
         FROM match AS sub
         WHERE main.season = sub.season
               AND main.country_id = sub.country_id);
```

## EXTRACT
Example: `EXTRACT(MONTH FROM date)`

## Nested Subqueries
```
SELECT
	-- Select the season and max goals scored in a match
	season,
    MAX(home_goal + away_goal) AS max_goals,
    -- Select the overall max goals scored in a match
   (SELECT MAX(home_goal + away_goal) FROM match) AS overall_max_goals,
   -- Select the max number of goals scored in any match in July
   (SELECT MAX(home_goal + away_goal) 
    FROM match
    WHERE id IN (
          SELECT id FROM match WHERE EXTRACT(MONTH FROM date) = 07)) AS july_max_goals
FROM match
GROUP BY season;
```

```
SELECT
	c.name AS country,
    -- Calculate the average matches per season
	AVG(matches) AS avg_seasonal_high_scores
FROM country AS c
-- Left join outer_s to country
LEFT JOIN (
  SELECT country_id, season,
         COUNT(id) AS matches
  FROM (
    SELECT country_id, season, id
	FROM match
	WHERE home_goal >= 5 OR away_goal >= 5) AS inner_s
  -- Close parentheses and alias the subquery
  GROUP BY country_id, season) AS outer_s
ON c.id = outer_s.country_id
GROUP BY country;
```

## Common Table Expressions
Used for referencing. Declared before main query.  CTEs are run once and stored in memory so it may be more efficient.

```
WITH
    s1 AS (
        SELECT country_id, id
        FROM match
        WHERE (home_goal + away_goal) >= 10
    ),
    s2 AS (
        SELECT country_id, id
        FROM match
        WHERE (home_goal + }away_goal) <= 1
    )
```

## Window Functions
Allows for both non-aggregated and aggregated values to be retrieved as columns.

## OVER()
```
SELECT
    date,
    (home_goal + away_goal) AS goals,
    AVG(home_goal + away_goal) OVER() AS overall_avg
FROM match
WHERE season = '2011/2012'
```

## RANK()
```
SELECT
    date,
    (home_goal + away_goal) AS goals,
    RANK() OVER(ORDER BY home_goal + away_goal) AS goals_rank
FROM match
WHERE season = '2011/2012'
```

## PARTITION BY
`AVG(home_goal) OVER(PARTITION BY season)`

## Sliding Windows
Perform calculations relative to the current row.  Can be used to calculate running totals, sums, averages, etc.  Can be partitioned by one or more columns.

`ROWS BETWEEN <start> AND <finish>`

Keywords include:
* `<#> PRECEDEING`
* `<#> FOLLOWING`
*  `UNBOUNDED PRECEDING`
*  `UNBOUNDED FOLLOWING`
*  `CURRENT ROW`


Sample code:
```
SELECT
    date,
    home_goal
    away_goal,
    SUM(home_goal)
        OVER(ORDER BY date ROWS BETWEEN
            UNBOUNDED PRECEDING AND CURRENT ROW) AS running_toal
FROM match
WHERE hometeam_id = 8456 AND season = '2011/2012';

```